<a href="https://colab.research.google.com/github/emmetorior/CN7030-/blob/main/ASSN_CN7050_FINAL1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [19]:
!pip install findspark
!pip install --upgrade pyspark pyarrow pandas
from google.colab import drive

In [20]:
import pandas as pd
import pyspark
import pyarrow as pa
import numpy as np
from pyspark.sql import SparkSession
from pyspark.sql.functions import array_contains
from pyspark.sql.functions import col, explode, split, array, lit
from pyspark.ml.feature import CountVectorizer, IDF
import ast


Check Files Exist and Are Readable.

1.   List item
2.   List item


```
# This is formatted as code
```



In [21]:
import os
parquet_validation_file = "/content/drive/MyDrive/Colab Notebooks/CN7050/validation-00000-of-00001.parquet"
print(os.path.exists(parquet_validation_file))

parquet_test_file = "/content/drive/MyDrive/Colab Notebooks/CN7050/test-00000-of-00001.parquet"
print(os.path.exists(parquet_test_file))

parquet_train_file = "/content/drive/MyDrive/Colab Notebooks/CN7050/train-00000-of-00001.parquet"
print(os.path.exists(parquet_train_file))

True
True
True


Load files into Spark. We have a train, test and validation file. Each one is loaded in from the parquet downloaded from huggingface.co

SET UP A SPARK SESSION AND SET OPTIONS

In [22]:
spark = SparkSession.Builder().master("local[*]").getOrCreate()
spark.conf.set("spark.sql.parquet.mergeSchema", "false")
spark.conf.set("spark.sql.parquet.binaryAsString", "true")

In [23]:

#spark.conf.set("spark.driver.memory", "8g")  # Increase to 8 GB (or higher if needed)
#spark.conf.set("spark.executor.memory", "8g")

#disable for now spark.conf.set("spark.sql.parquet.enableVectorizedReader", "false")

dfs_train = spark.read.format("parquet").option("mergeSchema", "false").load(parquet_train_file)
dfs_test = spark.read.format("parquet").option("mergeSchema", "false").load(parquet_test_file)
dfs_validation = spark.read.format("parquet").option("mergeSchema", "false").load(parquet_validation_file)

Check the file contents

In [24]:
dfs_train.show(5)
dfs_test.show(5)
dfs_validation.show(5)

+---------+---------+-----------+----------------+--------------------+--------------------+--------------------+
|       id|org_index|data_source|        industry|                text|              labels|         label_codes|
+---------+---------+-----------+----------------+--------------------+--------------------+--------------------+
|301972057|      600| Trustpilot|Price Comparison|My experience is ...|[[Staff support: ...|['staff-support.a...|
|301982453|      514|Google Play|         Banking|I love it so hand...|[[Company brand: ...|['company-brand.g...|
|301980653|      369|Google Play|    Ride Hailing|  Sometimes it takes|[[Company brand: ...|['company-brand.g...|
|301979991|      727|Apple Store|         Fashion|This is the worst...|[[Logistics rides...|['logistics-rides...|
|301984330|      549|Google Play|  Travel Booking|So easy & loads o...|[[Company brand: ...|['company-brand.g...|
+---------+---------+-----------+----------------+--------------------+-----------------

File was not loading. Implemented a Workaround by converting the pandas version

1.   List item
2.   List item

of the parquet file converted to CSV and read into spark to create a spark dataframe. Maybe delete this segment later as problem was resolved.  

*italicized text*

In [25]:
print("Training file row count:", dfs_train.count())
dfs_train.printSchema()
print("Testing file row count:", dfs_test.count())
dfs_test.printSchema()
print("Validation file row count:", dfs_validation.count())
dfs_validation.printSchema()

Training file row count: 7930
root
 |-- id: long (nullable = true)
 |-- org_index: long (nullable = true)
 |-- data_source: string (nullable = true)
 |-- industry: string (nullable = true)
 |-- text: string (nullable = true)
 |-- labels: array (nullable = true)
 |    |-- element: array (containsNull = true)
 |    |    |-- element: string (containsNull = true)
 |-- label_codes: string (nullable = true)

Testing file row count: 1587
root
 |-- id: long (nullable = true)
 |-- org_index: long (nullable = true)
 |-- data_source: string (nullable = true)
 |-- industry: string (nullable = true)
 |-- text: string (nullable = true)
 |-- labels: array (nullable = true)
 |    |-- element: array (containsNull = true)
 |    |    |-- element: string (containsNull = true)
 |-- label_codes: string (nullable = true)

Validation file row count: 1057
root
 |-- id: long (nullable = true)
 |-- org_index: long (nullable = true)
 |-- data_source: string (nullable = true)
 |-- industry: string (nullable = true

In [26]:
#findspark.init()
from pyspark.sql.functions import udf
from pyspark.sql.types import ArrayType, StringType
from pyspark.sql.functions import col, sum


In [27]:
null_counts_train = dfs_train.select([sum(col(c).isNull().cast("int")).alias(c) for c in dfs_train.columns])
null_counts_train.show()
null_counts_test = dfs_test.select([sum(col(c).isNull().cast("int")).alias(c) for c in dfs_test.columns])
null_counts_test.show()
null_counts_validation = dfs_validation.select([sum(col(c).isNull().cast("int")).alias(c) for c in dfs_validation.columns])
null_counts_validation.show()

+---+---------+-----------+--------+----+------+-----------+
| id|org_index|data_source|industry|text|labels|label_codes|
+---+---------+-----------+--------+----+------+-----------+
|  0|        0|          0|       0|   0|     0|          0|
+---+---------+-----------+--------+----+------+-----------+

+---+---------+-----------+--------+----+------+-----------+
| id|org_index|data_source|industry|text|labels|label_codes|
+---+---------+-----------+--------+----+------+-----------+
|  0|        0|          0|       0|   0|     0|          0|
+---+---------+-----------+--------+----+------+-----------+

+---+---------+-----------+--------+----+------+-----------+
| id|org_index|data_source|industry|text|labels|label_codes|
+---+---------+-----------+--------+----+------+-----------+
|  0|        0|          0|       0|   0|     0|          0|
+---+---------+-----------+--------+----+------+-----------+



In [28]:
# ACD-Only Classifier Pipeline
from pyspark.sql.functions import col, array_contains, lit, udf
from pyspark.ml.feature import VectorAssembler
from pyspark.ml.classification import LogisticRegression
from pyspark.ml.evaluation import MulticlassClassificationEvaluator
import ast

In [29]:
#Cleaning and Extracting functions
def clean_categories(categories):
    return categories.replace('-', '_').replace('.', '_')

@udf(returnType=ArrayType(StringType()))
def extract_categories(label_codes):
    if isinstance(label_codes, str):
        codes = ast.literal_eval(label_codes)
        categories = [code.rsplit('.', 1)[0] for code in codes]
        return [clean_categories(cat) for cat in categories]
    return []

# Step 1: Extract Categories

In [30]:
#dfs_train_clean = dfs_train_clean.limit(5000)
#
# Extract categories for each DataFrame
dfs_train_clean = dfs_train.withColumn("categories", extract_categories(col("label_codes")))
dfs_train_clean = dfs_train_clean.limit(80)

dfs_test_clean = dfs_test.withColumn("categories", extract_categories(col("label_codes")))
dfs_test_clean = dfs_test_clean.limit(80)

dfs_validation_clean = dfs_validation.withColumn("categories", extract_categories(col("label_codes")))
dfs_validation_clean = dfs_validation_clean.limit(80)

In [31]:
# Generate binary indicator columns for each category
all_categories = dfs_train_clean.select(explode("categories")).distinct().rdd.flatMap(lambda x: x).collect()

for category in all_categories:
    clean_cat = category.replace('-', '_').replace('.', '_')
    dfs_train_clean = dfs_train_clean.withColumn(f"has_{clean_cat}", array_contains(col("categories"), category).cast("integer"))
    dfs_test_clean = dfs_test_clean.withColumn(f"has_{clean_cat}", array_contains(col("categories"), category).cast("integer"))
    dfs_validation_clean = dfs_validation_clean.withColumn(f"has_{clean_cat}", array_contains(col("categories"), category).cast("integer"))

# Cache DataFrames to avoid recomputation
dfs_train_clean.cache()
dfs_test_clean.cache()
dfs_validation_clean.cache()

DataFrame[id: bigint, org_index: bigint, data_source: string, industry: string, text: string, labels: array<array<string>>, label_codes: string, categories: array<string>, has_staff_support_attitude_of_staff: int, has_company_brand_reviews: int, has_company_brand_general_satisfaction: int, has_company_brand_competitor: int, has_logistics_rides_speed: int, has_online_experience_app_website: int, has_purchase_booking_experience_ease_of_use: int, has_value_price_value_for_money: int, has_value_discounts_promotions: int, has_account_management_account_access: int, has_staff_support_phone: int]

In [32]:
# Generate binary indicator columns for each category
all_categories = dfs_train_clean.select(explode("categories")).distinct().rdd.flatMap(lambda x: x).collect()

for category in all_categories:
    clean_cat = category.replace('-', '_').replace('.', '_')
    dfs_train_clean = dfs_train_clean.withColumn(f"has_{clean_cat}", array_contains(col("categories"), category).cast("integer"))
    dfs_test_clean = dfs_test_clean.withColumn(f"has_{clean_cat}", array_contains(col("categories"), category).cast("integer"))
    dfs_validation_clean = dfs_validation_clean.withColumn(f"has_{clean_cat}", array_contains(col("categories"), category).cast("integer"))


In [33]:
dfs_train_clean.show(5)
dfs_test_clean.show(5)

# Cache DataFrames to avoid recomputation
dfs_train_clean.cache()
dfs_test_clean.cache()
dfs_validation_clean.cache()

dfs_train_clean.select("data_source", "categories").show(20, truncate=False)

# Select feature columns
feature_cols = [f"has_{cat.replace('-', '_').replace('.', '_')}" for cat in all_categories]

dfs_train_clean.select("data_source", "categories").show(20, truncate=False)


+---------+---------+-----------+----------------+--------------------+--------------------+--------------------+--------------------+-----------------------------------+-------------------------+--------------------------------------+----------------------------+-------------------------+---------------------------------+-------------------------------------------+-------------------------------+------------------------------+-------------------------------------+-----------------------+
|       id|org_index|data_source|        industry|                text|              labels|         label_codes|          categories|has_staff_support_attitude_of_staff|has_company_brand_reviews|has_company_brand_general_satisfaction|has_company_brand_competitor|has_logistics_rides_speed|has_online_experience_app_website|has_purchase_booking_experience_ease_of_use|has_value_price_value_for_money|has_value_discounts_promotions|has_account_management_account_access|has_staff_support_phone|
+---------+-

In [34]:
# SKIP
# 3. Train Classifiers
#models = {}
#for category in all_categories:
#    lr = LogisticRegression(featuresCol="features", labelCol=category)
#    pipeline = Pipeline(stages=[tokenizer, remover, cv, idf, lr])
#    models[category] = pipeline.fit(df)#
#
## 4. Test on New Data (new_text_df)
#for category, model in models.items():
#    predictions = model.transform(new_text_df)
#    predictions.select("text", category, "prediction").show() #show the text, the real category, and the prediction.

In [38]:
from pyspark.sql.functions import col, array_contains, lit
from pyspark.ml.feature import VectorAssembler
from pyspark.ml.classification import LinearSVC
from pyspark.ml.evaluation import MulticlassClassificationEvaluator
import ast
import numpy as np
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense, Dropout, Conv1D, GlobalMaxPooling1D


In [41]:
# Train Logistic Regression Model with Regularization and Limited Iterations
assembler = VectorAssembler(inputCols=feature_cols, outputCol="features")
train_data = assembler.transform(dfs_train_clean).select("features", "text")
train_data.show(5)


+--------------------+--------------------+
|            features|                text|
+--------------------+--------------------+
|(11,[0,1,2],[1.0,...|My experience is ...|
|(11,[2,3],[1.0,1.0])|I love it so hand...|
|      (11,[2],[1.0])|  Sometimes it takes|
|(11,[3,4,5],[1.0,...|This is the worst...|
|      (11,[2],[1.0])|So easy & loads o...|
+--------------------+--------------------+
only showing top 5 rows



In [42]:
# Train SVM Model
assembler = VectorAssembler(inputCols=feature_cols, outputCol="features")
train_data = assembler.transform(dfs_train_clean).select("features", "id")

svm = LinearSVC(featuresCol="features", labelCol="id", maxIter=20, regParam=0.1)
svm_model = svm.fit(train_data)

# Test on test set
test_data = assembler.transform(dfs_test_clean).select("features", "id")
svm_predictions = svm_model.transform(test_data)

# Evaluate SVM
svm_evaluator = MulticlassClassificationEvaluator(labelCol="id", predictionCol="prediction", metricName="f1")

f1_svm = svm_evaluator.evaluate(svm_predictions)
precision_svm = svm_evaluator.evaluate(svm_predictions, {svm_evaluator.metricName: "weightedPrecision"})
recall_svm = svm_evaluator.evaluate(svm_predictions, {svm_evaluator.metricName: "weightedRecall"})

print(f"SVM F1 Score: {f1_svm}")
print(f"SVM Precision: {precision_svm}")
print(f"SVM Recall: {recall_svm}")

# Convert to Pandas for LSTM and CNN
train_pandas = train_data.toPandas()
test_pandas = test_data.toPandas()

X_train = train_pandas[feature_cols].values
X_test = test_pandas[feature_cols].values

y_train = train_pandas['id'].values
y_test = test_pandas['id'].values

# Reshape for LSTM and CNN
X_train = X_train[..., np.newaxis]
X_test = X_test[..., np.newaxis]

# LSTM Model
lstm_model = Sequential([
    LSTM(64, input_shape=(X_train.shape[1], 1)),
    Dropout(0.2),
    Dense(len(np.unique(y_train)), activation='softmax')
])

lstm_model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])
lstm_model.fit(X_train, y_train, epochs=5, batch_size=32, verbose=0)

lstm_loss, lstm_acc = lstm_model.evaluate(X_test, y_test, verbose=0)

print(f"LSTM Accuracy: {lstm_acc}")

# CNN Model
cnn_model = Sequential([
    Conv1D(64, 3, activation='relu', input_shape=(X_train.shape[1], 1)),
    GlobalMaxPooling1D(),
    Dense(64, activation='relu'),
    Dense(len(np.unique(y_train)), activation='softmax')
])

cnn_model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])
cnn_model.fit(X_train, y_train, epochs=5, batch_size=32, verbose=0)

cnn_loss, cnn_acc = cnn_model.evaluate(X_test, y_test, verbose=0)

print(f"CNN Accuracy: {cnn_acc}")

# Clear cache to free memory
dfs_train_clean.unpersist()
dfs_test_clean.unpersist()
dfs_validation_clean.unpersist()


Py4JJavaError: An error occurred while calling o1299.fit.
: org.apache.spark.SparkException: Job aborted due to stage failure: Task 0 in stage 77.0 failed 1 times, most recent failure: Lost task 0.0 in stage 77.0 (TID 63) (e54f9cbed778 executor driver): java.lang.RuntimeException: Labels MUST be in {0, 1}, but got 3.01972057E8
	at org.apache.spark.sql.catalyst.expressions.GeneratedClass$GeneratedIteratorForCodegenStage1.project_doConsume_0$(Unknown Source)
	at org.apache.spark.sql.catalyst.expressions.GeneratedClass$GeneratedIteratorForCodegenStage1.processNext(Unknown Source)
	at org.apache.spark.sql.execution.BufferedRowIterator.hasNext(BufferedRowIterator.java:43)
	at org.apache.spark.sql.execution.WholeStageCodegenEvaluatorFactory$WholeStageCodegenPartitionEvaluator$$anon$1.hasNext(WholeStageCodegenEvaluatorFactory.scala:43)
	at scala.collection.Iterator$$anon$10.hasNext(Iterator.scala:460)
	at scala.collection.Iterator$$anon$10.hasNext(Iterator.scala:460)
	at scala.collection.Iter

In [36]:

#lr = LogisticRegression(featuresCol="features", labelCol="id", maxIter=20, regParam=0.1)
#model = lr.fit(train_data)

Py4JJavaError: An error occurred while calling o1077.fit.
: java.lang.OutOfMemoryError: Java heap space
	at scala.reflect.ManifestFactory$DoubleManifest.newArray(Manifest.scala:194)
	at scala.reflect.ManifestFactory$DoubleManifest.newArray(Manifest.scala:191)
	at scala.Array$.ofDim(Array.scala:305)
	at org.apache.spark.ml.stat.MultiClassSummarizer.histogram(MultiClassSummarizer.scala:89)
	at org.apache.spark.ml.classification.LogisticRegression.$anonfun$train$1(LogisticRegression.scala:520)
	at org.apache.spark.ml.classification.LogisticRegression$$Lambda$4092/0x00000008417cb840.apply(Unknown Source)
	at org.apache.spark.ml.util.Instrumentation$.$anonfun$instrumented$1(Instrumentation.scala:191)
	at org.apache.spark.ml.util.Instrumentation$$$Lambda$4093/0x00000008417cb440.apply(Unknown Source)
	at scala.util.Try$.apply(Try.scala:213)
	at org.apache.spark.ml.util.Instrumentation$.instrumented(Instrumentation.scala:191)
	at org.apache.spark.ml.classification.LogisticRegression.train(Logi

In [ ]:




# Test on test set
test_data = assembler.transform(dfs_test_clean).select("features", "id")
predictions = model.transform(test_data)

# Evaluate model
evaluator = MulticlassClassificationEvaluator(labelCol="id", predictionCol="prediction", metricName="f1")

f1 = evaluator.evaluate(predictions)
precision = evaluator.evaluate(predictions, {evaluator.metricName: "weightedPrecision"})
recall = evaluator.evaluate(predictions, {evaluator.metricName: "weightedRecall"})

print(f"F1 Score: {f1}")
print(f"Precision: {precision}")
print(f"Recall: {recall}")

# Validate on validation set
validation_data = assembler.transform(dfs_validation_clean).select("features", "id")
validation_predictions = model.transform(validation_data)

f1_val = evaluator.evaluate(validation_predictions)
precision_val = evaluator.evaluate(validation_predictions, {evaluator.metricName: "weightedPrecision"})
recall_val = evaluator.evaluate(validation_predictions, {evaluator.metricName: "weightedRecall"})

print(f"Validation F1 Score: {f1_val}")
print(f"Validation Precision: {precision_val}")
print(f"Validation Recall: {recall_val}")

# Clear cache to free memory
dfs_train_clean.unpersist()
dfs_test_clean.unpersist()
dfs_validation_clean.unpersist()

In [ ]:

# Generate binary indicator columns for each category
all_categories = dfs_train_clean.select(explode("categories")).distinct().rdd.flatMap(lambda x: x).collect()

for category in all_categories:
    clean_cat = category.replace('-', '_').replace('.', '_')
    dfs_train_clean = dfs_train_clean.withColumn(f"has_{clean_cat}", array_contains(col("categories"), category).cast("integer"))
    dfs_test_clean = dfs_test_clean.withColumn(f"has_{clean_cat}", array_contains(col("categories"), category).cast("integer"))
    dfs_validation_clean = dfs_validation_clean.withColumn(f"has_{clean_cat}", array_contains(col("categories"), category).cast("integer"))

# Select feature columns
feature_cols = [f"has_{cat.replace('-', '_').replace('.', '_')}" for cat in all_categories]

# Train Logistic Regression Model
assembler = VectorAssembler(inputCols=feature_cols, outputCol="features")
train_data = assembler.transform(dfs_train_clean).select("features", "id")

lr = LogisticRegression(featuresCol="features", labelCol="id")
model = lr.fit(train_data)

# Test on test set
test_data = assembler.transform(dfs_test_clean).select("features", "id")
predictions = model.transform(test_data)

# Evaluate model
evaluator = MulticlassClassificationEvaluator(labelCol="id", predictionCol="prediction", metricName="f1")

f1 = evaluator.evaluate(predictions)
precision = evaluator.evaluate(predictions, {evaluator.metricName: "weightedPrecision"})
recall = evaluator.evaluate(predictions, {evaluator.metricName: "weightedRecall"})

print(f"F1 Score: {f1}")
print(f"Precision: {precision}")
print(f"Recall: {recall}")

# Validate on validation set
validation_data = assembler.transform(dfs_validation_clean).select("features", "id")
validation_predictions = model.transform(validation_data)

f1_val = evaluator.evaluate(validation_predictions)
precision_val = evaluator.evaluate(validation_predictions, {evaluator.metricName: "weightedPrecision"})
recall_val = evaluator.evaluate(validation_predictions, {evaluator.metricName: "weightedRecall"})

print(f"Validation F1 Score: {f1_val}")
print(f"Validation Precision: {precision_val}")
print(f"Validation Recall: {recall_val}")

In [ ]:
# Add me back later, if needed. Initialize Spark session. Display the schema to understand the data structure
#dfp_train.printSchema()  Let's extract the categories from label_codes. First, we need to convert the string representation of label codes to actual lists. Then We'll use a UDF (User Defined Function) for this
def clean_categories(categories):
  return categories.replace('-', '_').replace('.', '_')

#Extract just the category part from each label code
@udf(returnType=ArrayType(StringType()))
def extract_categories(label_codes):

    # Convert string representation to list
    if isinstance(label_codes, str):
        codes = ast.literal_eval(label_codes)#

        # Extract category part (remove the sentiment indicator at the end)
        categories = [code.rsplit('.', 1)[0] for code in codes]

        categories = clean_categories(categories)
        return categories
    return []

In [ ]:
import ast

# Convert 'label_codes' from string to list
dfs_train['label_codes'] = dfs_train['label_codes'].apply(
    lambda x: ast.literal_eval(x) if isinstance(x, str) else x
)

# Extract categories and clean them
def extract_categories(label_codes):
    return [
        code.rsplit('.', 1)[0].replace('-', '_').replace('.', '_')
        for code in label_codes
    ]

# Apply the function to create a 'categories' column
dfp_train['categories'] = dfp_train['label_codes'].apply(extract_categories)

In [ ]:

#dfs_train = spark.createDataFrame(dfp_train)     #pandas df has no ... withColumn so we need to convert it - Oops it's not a pandas df, it's a pyarrow Table that needs to be first converted to a pandas df
#dfp_train = dfp_train.to_pandas()
#dfs_train = spark.createDataFrame(dfp_train)
##It seems that the to_pandas method is not available for Spark DataFrames -  use the collect method to convert the Spark DataFrame to a Pandas DataFrame:

# Convert Spark DataFrame to Pandas
dfp_train = spark.read.parquet(parquet_train_file).toPandas()
print ( dfp_train.columns)
# For each category, create a binary indicator column (1 if present, 0 if not)
print(dfp_train.head(10))



In [ ]:
## Add me back later, if needed. Initialize Spark session. Display the schema to understand the data structure
##dfp_train.printSchema()  Let's extract the categories from label_codes. First, we need to convert the string representation of label codes to actual lists. Then We'll use a UDF (User Defined Function) for this
#def clean_categories(categories):
#  return categories.replace('-', '_').replace('.', '_')
#
##Extract just the category part from each label code
#@udf(returnType=ArrayType(StringType()))
#def extract_categories(label_codes):
#
#    # Convert string representation to list
#    if isinstance(label_codes, str):
#        codes = ast.literal_eval(label_codes)#
#
#        # Extract category part (remove the sentiment indicator at the end)
#        categories = [code.rsplit('.', 1)[0] for code in codes]
#
#        categories = clean_categories(categories)
#        return categories
#    return []

In [ ]:
#df_with_categories = dfs_train.withColumn("categories", extract_categories(col("label_codes")))
# Create a list of all unique categories from the dataset
df_with_categories.printSchema()
#all_categories = df_with_categories.select(explode("categories")).distinct().collect()
#all_categories = [row[0] for row in all_categories]


In [ ]:
# ACD + ABSA Full Pipeline (SVM, LSTM, and CNN Models) - Optimized for Speed

from pyspark.sql.functions import col, array_contains
from pyspark.ml.feature import Tokenizer, HashingTF, IDF
from pyspark.ml.classification import LinearSVC
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense, Dropout, Conv1D, GlobalMaxPooling1D
import numpy as np
import tensorflow as tf
import matplotlib.pyplot as plt
import ast

# Step 1: Aspect Category Detection (ACD) with Clean Column Names
for aspect in all_categories:
    clean_aspect = aspect.replace('-', '_').replace('.', '_')
    df_with_categories = df_with_categories.withColumn(
        f"has_{clean_aspect}", array_contains(col("categories"), aspect).cast("integer")
    )

# Step 2: Extract Sentiment for Each Category

# Extract 'label_codes' from the original Spark DataFrame
#label_codes_df = df_with_categories.select('id', 'label_codes').toPandas()
label_codes_df = df_with_categories.select('id', 'label_codes', 'categories').toPandas()
print(label_codes_df)

# Merge with train_pandas using the 'id' column
train_pandas = train_pandas.merge(label_codes_df, on='id', how='left')

def extract_aspect_and_sentiment(label_codes):
    if isinstance(label_codes, str):
        codes = ast.literal_eval(label_codes)
        return [(code.rsplit('.', 1)[0], int(code.rsplit('.', 1)[-1])) for code in codes]
    return []

train_pandas['aspect_sentiment_pairs'] = train_pandas['label_codes'].apply(extract_aspect_and_sentiment)

# For each category, create a sentiment column (-1, 0, or 1)
for aspect in all_categories:
    clean_aspect = aspect.replace('-', '_').replace('.', '_')
    train_pandas[f"sentiment_{clean_aspect}"] = train_pandas['aspect_sentiment_pairs'].apply(
        lambda x: next((sent for asp, sent in x if asp == aspect), 0)
    )

# Step 3: Convert to Pandas for Deep Learning
#train_pandas = train_data.limit(1000).toPandas()  # Limit data for speed
#test_pandas = test_data.limit(200).toPandas()
train_pandas = df_with_categories.limit(1000).toPandas()
test_pandas = df_with_categories.limit(200).toPandas()

# Convert sparse vector to numpy array
def sparse_to_array(v):
    return v.toArray() if hasattr(v, "toArray") else v

train_pandas['features_array'] = train_pandas['features'].apply(sparse_to_array)
test_pandas['features_array'] = test_pandas['features'].apply(sparse_to_array)

# Step 4: LSTM Model for ABSA with Reduced Complexity
X_train = np.stack(train_pandas['features_array'].values)
y_train = train_pandas[label_cols].values

X_test = np.stack(test_pandas['features_array'].values)
y_test = test_pandas[label_cols].values

lstm_model = Sequential([
    LSTM(64, input_shape=(X_train.shape[1], 1), return_sequences=False),  # Reduced LSTM units
    Dropout(0.2),
    Dense(len(label_cols), activation='sigmoid')
])

lstm_model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])
history_lstm = lstm_model.fit(X_train, y_train, epochs=5, batch_size=32, verbose=0)  # Fewer epochs

# Step 5: CNN Model for ABSA with Faster Conv1D
cnn_model = Sequential([
    Conv1D(32, 3, activation='relu', input_shape=(X_train.shape[1], 1)),  # Reduced filters
    GlobalMaxPooling1D(),
    Dense(len(label_cols), activation='sigmoid')
])

cnn_model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])
history_cnn = cnn_model.fit(X_train, y_train, epochs=5, batch_size=32, verbose=0)

# Step 6: Compare Models
svm_model = train_absa_svm("battery_life", df_with_categories)

print("SVM Model Trained")
print("LSTM Accuracy:", lstm_model.evaluate(X_test, y_test, verbose=0))
print("CNN Accuracy:", cnn_model.evaluate(X_test, y_test, verbose=0))

# Step 7: Visualizations
plt.figure(figsize=(10, 5))

# Plot LSTM Loss
plt.subplot(1, 2, 1)
plt.plot(history_lstm.history['loss'], label='LSTM Loss')
plt.title('LSTM Loss Over Epochs')
plt.xlabel('Epochs')
plt.ylabel('Loss')
plt.legend()

# Plot CNN Accuracy
plt.subplot(1, 2, 2)
plt.plot(history_cnn.history['accuracy'], label='CNN Accuracy')
plt.title('CNN Accuracy Over Epochs')
plt.xlabel('Epochs')
plt.ylabel('Accuracy')
plt.legend()

plt.tight_layout()
plt.show()


PRINT ALL CATEGORIES FOUND

In [ ]:
for cat in all_categories:
  print(cat + '\n')

In [ ]:
# For each category, create a binary indicator column (1 if present, 0 if not)
for cat in all_categories:
    df_with_categories = df_with_categories.withColumn(
        f"has_{cat.replace('-', '_').replace('.', '_')}",
        array_contains(col("categories"), cat).cast("integer")
    )


In [ ]:
for cat in all_categories:
  print(cat + '\n')

In [ ]:
print(train_pandas.columns)

In [ ]:
#
FIX ME TODAY
# ACD + ABSA Full Pipeline (SVM, LSTM, and CNN Models) - Optimized for Speed

from pyspark.sql.functions import col, array_contains
from pyspark.ml.feature import Tokenizer, HashingTF, IDF
from pyspark.ml.classification import LinearSVC
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense, Dropout, Conv1D, GlobalMaxPooling1D
import numpy as np
import tensorflow as tf
import matplotlib.pyplot as plt
import ast

# Step 1: Aspect Category Detection (ACD) with Clean Column Names
for aspect in all_categories:
    clean_aspect = aspect.replace('-', '_').replace('.', '_')
    df_with_categories = df_with_categories.withColumn(
        f"has_{clean_aspect}", array_contains(col("categories"), aspect).cast("integer")
    )

# Step 2: Extract Sentiment for Each Category

def extract_aspect_and_sentiment(label_codes):
    if isinstance(label_codes, str):
        codes = ast.literal_eval(label_codes)
        return [(code.rsplit('.', 1)[0], int(code.rsplit('.', 1)[-1])) for code in codes]
    return []

train_pandas['aspect_sentiment_pairs'] = train_pandas['label_codes'].apply(extract_aspect_and_sentiment)

# For each category, create a sentiment column (-1, 0, or 1)
for aspect in all_categories:
    clean_aspect = aspect.replace('-', '_').replace('.', '_')
    train_pandas[f"sentiment_{clean_aspect}"] = train_pandas['aspect_sentiment_pairs'].apply(
        lambda x: next((sent for asp, sent in x if asp == aspect), 0)
    )

# Step 3: Convert to Pandas for Deep Learning
train_pandas = train_data.limit(1000).toPandas()  # Limit data for speed
test_pandas = test_data.limit(200).toPandas()

# Convert sparse vector to numpy array
def sparse_to_array(v):
    return v.toArray() if hasattr(v, "toArray") else v

train_pandas['features_array'] = train_pandas['features'].apply(sparse_to_array)
test_pandas['features_array'] = test_pandas['features'].apply(sparse_to_array)

# Step 4: LSTM Model for ABSA with Reduced Complexity
X_train = np.stack(train_pandas['features_array'].values)
y_train = train_pandas[label_cols].values

X_test = np.stack(test_pandas['features_array'].values)
y_test = test_pandas[label_cols].values

lstm_model = Sequential([
    LSTM(64, input_shape=(X_train.shape[1], 1), return_sequences=False),  # Reduced LSTM units
    Dropout(0.2),
    Dense(len(label_cols), activation='sigmoid')
])

lstm_model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])
history_lstm = lstm_model.fit(X_train, y_train, epochs=5, batch_size=32, verbose=0)  # Fewer epochs

# Step 5: CNN Model for ABSA with Faster Conv1D
cnn_model = Sequential([
    Conv1D(32, 3, activation='relu', input_shape=(X_train.shape[1], 1)),  # Reduced filters
    GlobalMaxPooling1D(),
    Dense(len(label_cols), activation='sigmoid')
])

cnn_model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])
history_cnn = cnn_model.fit(X_train, y_train, epochs=5, batch_size=32, verbose=0)

# Step 6: Compare Models
svm_model = train_absa_svm("battery_life", df_with_categories)

print("SVM Model Trained")
print("LSTM Accuracy:", lstm_model.evaluate(X_test, y_test, verbose=0))
print("CNN Accuracy:", cnn_model.evaluate(X_test, y_test, verbose=0))

# Step 7: Visualizations
plt.figure(figsize=(10, 5))

# Plot LSTM Loss
plt.subplot(1, 2, 1)
plt.plot(history_lstm.history['loss'], label='LSTM Loss')
plt.title('LSTM Loss Over Epochs')
plt.xlabel('Epochs')
plt.ylabel('Loss')
plt.legend()

# Plot CNN Accuracy
plt.subplot(1, 2, 2)
plt.plot(history_cnn.history['accuracy'], label='CNN Accuracy')
plt.title('CNN Accuracy Over Epochs')
plt.xlabel('Epochs')
plt.ylabel('Accuracy')
plt.legend()

plt.tight_layout()
plt.show()


Tokenize the user comments from the text column and remove stop words.

In [ ]:

from pyspark.ml.feature import Tokenizer, StopWordsRemover

# Select relevant columns for the ACD task
feature_cols = [f"has_{category.replace('-', '_').replace('.', '_')}" for category in all_categories]
acd_df = df_with_categories.select("id", "text", *feature_cols)

# Convert text to features using TF-IDF
# First, tokenize the text
from pyspark.ml.feature import Tokenizer, StopWordsRemover

tokenizer = Tokenizer(inputCol="text", outputCol="words")
wordsData = tokenizer.transform(acd_df)

# Remove stop words
remover = StopWordsRemover(inputCol="words", outputCol="filtered_words")
filtered_data = remover.transform(wordsData)

# Convert words to term frequency features
cv = CountVectorizer(inputCol="filtered_words", outputCol="tf", minDF=2.0)
cv_model = cv.fit(filtered_data)
tf_data = cv_model.transform(filtered_data)

# Convert term frequency features to TF-IDF
idf = IDF(inputCol="tf", outputCol="features")
idf_model = idf.fit(tf_data)
tfidf_data = idf_model.transform(tf_data)

# Final dataset ready for modeling
final_acd_df = tfidf_data.select("id", "features", *feature_cols)

# Split the data into training and testing sets
train_data, test_data = final_acd_df.randomSplit([0.8, 0.2], seed=42)

# Cache the datasets for faster processing
train_data.cache()
test_data.cache()


In [ ]:
#final_acd_df.show(5)


In [ ]:
#Since you're working with text data, you'll need to extract relevant features from each document. Some common techniques include:
#Tokenization (splitting text into individual words or tokens)
#Stopword removal (removing common words like "the", "and", etc.)
#Stemming or Lemmatization (reducing words to their base form)
#Part-of-speech tagging (identifying grammatical categories of each word)
# Create a SparkSession with your test/train/validate data sets
#spark = SparkSession.builder.appName("Aspect-Based Sentiment Analysis").getOrCreate()

# Load the training and validate datasets into DataFrames
#train_df = spark.read.csv("path/to/train/dataset", header=True, inferSchema=True)
#val_df = spark.read.csv("path/to/val/dataset", header=True, inferSchema=True)

# Define a function to extract features from each column in the dataset
def feature_extraction(df):
    # Tokenization and stopword removal
    df = df.select(
        explode(col("text").cast("string")).alias("tokens"),
        explode(col("sentiment").cast("string")).alias("sentiment")
    )

    # Stemming or Lemmatization
    df = df.withColumn(
        "stemmed_tokens",
        df.tokens.map(lambda x: x.lower() if x.isnumeric() else x)  # Remove numbers and convert to lowercase
    )

    # Part-of-speech tagging (optional)
    #df = df.withColumn("pos_tags", explode(col("word").cast("string")).map(lambda x: {"POS": "NNP"} if x.isnumeric() else {"POS": "NOUN"})  # Example

    return df

In [ ]:
from collections import Counter
import matplotlib.pyplot as plt
import pandas as pd

#all_words = filtered_data.rdd.flatMap(lambda x: x.filtered_words).collect()
all_words = filtered_data.select("filtered_words").rdd.flatMap(lambda x: x.filtered_words).collect()
word_counts = Counter(all_words)
top_words = dict(word_counts.most_common(20))

plt.figure(figsize=(12, 6))
plt.bar(top_words.keys(), top_words.values())
plt.xticks(rotation=45, ha='right')
plt.xlabel("Words")
plt.ylabel("Frequency")
plt.title("Top 20 Word Frequency")
plt.tight_layout()
plt.show()
#2. TF-IDF Visualization:


In [ ]:
import seaborn as sns
import pandas as pd

# Extract only the binary category columns and sentiment
category_columns = [col for col in train_pandas.columns if col.startswith('has_')]
df_for_heatmap = train_pandas[category_columns + ['sentiment']]

# Compute correlation matrix
correlation_matrix = df_for_heatmap.corr()

# Plot heatmap
plt.figure(figsize=(10, 8))
sns.heatmap(correlation_matrix, annot=True, cmap='coolwarm', fmt=".2f")
plt.title('Aspect-Sentiment Correlation Heatmap')
plt.show()

In [ ]:

#Chart: Heatmap or bar chart.
#Purpose: Shows the TF-IDF scores for selected words across documents.
#Implementation: After TF-IDF, convert the sparse vectors to dense arrays and plot the values.
#Python

import numpy as np
import seaborn as sns

# Assuming you have a few documents and words you want to visualize
selected_docs = tfidf_data.take(5) #take the first 5 documents.
selected_words = cv_model.vocabulary[:10] # take the first 10 vocabulary words.

tfidf_matrix = np.zeros((len(selected_docs), len(selected_words)))

for i, row in enumerate(selected_docs):
    dense_features = row.features.toArray()
    for j, word in enumerate(selected_words):
        if word in cv_model.vocabulary:
            word_index = cv_model.vocabulary.index(word)
            tfidf_matrix[i,j] = dense_features[word_index]

plt.figure(figsize=(12,8))
sns.heatmap(tfidf_matrix, annot=True, xticklabels=selected_words)
plt.xlabel("Selected Words")
plt.ylabel("Documents")
plt.title("TF-IDF Heatmap")
plt.show()

#3. Model Evaluation Metrics:
#
#Charts: Confusion matrix, ROC curve, precision-recall curve.
#Purpose: Shows the performance of the trained classification model.
#Implementation: After training and predicting on the test set, use metrics like accuracy, precision, recall, F1-score, and create plots to visualize them.
#Confusion Matrix: shows the number of correct and incorrect predictions for each class.
#ROC Curve: shows the trade-off between true positive rate and false positive rate.
#Precision-Recall Curve: shows the trade-off between precision and recall.
#4. Document Length Distribution:#
#
#Chart: Histogram.
#Purpose: Shows the distribution of the number of words in each document.
#Implementation: Count the number of words in each document after tokenization.
#Python#
#
doc_lengths = wordsData.rdd.map(lambda x: len(x.words)).collect()

plt.figure(figsize=(8, 6))
plt.hist(doc_lengths, bins=50)
plt.xlabel("Document Length (Number of Words)")
plt.ylabel("Frequency")
plt.title("Document Length Distribution")
plt.show()

In [ ]:
train_data.show(10)

In [ ]:
from pyspark.ml.classification import LinearSVC
from pyspark.ml.evaluation import BinaryClassificationEvaluator, MulticlassClassificationEvaluator
from pyspark.sql.functions import col

# Function to train and evaluate an SVM model for one category
def train_svm_for_category(category, train_data, test_data):
    # Prepare label column name
    label_col = f"has_{category.replace('-', '_').replace('.', '_')}"

    # Create SVM model
    svm = LinearSVC(featuresCol="features", labelCol=label_col, maxIter=10)

    # Train the model
    model = svm.fit(train_data)

    # Make predictions
    predictions = model.transform(test_data)

    # For binary classification metrics (precision, recall)
    binary_evaluator = BinaryClassificationEvaluator(
        labelCol=label_col, rawPredictionCol="rawPrediction")

    # For F1 score (use MulticlassClassificationEvaluator)
    multi_evaluator = MulticlassClassificationEvaluator(
        labelCol=label_col, predictionCol="prediction", metricName="f1")

    # Calculate AUC (Area Under ROC)
    auc = binary_evaluator.evaluate(predictions)

    # Calculate F1
    f1 = multi_evaluator.evaluate(predictions)

    # Calculate precision and recall manually
    # First, get true positives, false positives, true negatives, false negatives
    tp = predictions.filter((col(label_col) == 1) & (col("prediction") == 1)).count()
    fp = predictions.filter((col(label_col) == 0) & (col("prediction") == 1)).count()
    fn = predictions.filter((col(label_col) == 1) & (col("prediction") == 0)).count()

    # Calculate precision and recall
    precision = tp / (tp + fp) if (tp + fp) > 0 else 0
    recall = tp / (tp + fn) if (tp + fn) > 0 else 0

    return {
        "category": category,
        "precision": precision,
        "recall": recall,
        "f1": f1,
        "auc": auc
    }

Models were running too slow. \Caching the DataFrame during SVM training to avoid redundant data processing. Reducing the number of SVM iterations and limiting the dataset size for deep learning. Simplifying the LSTM and CNN architectures by reducing the number of units and filters. Lowering the number of epochs and silencing verbose outputs for faster training.

Added visualizations to pipeline:

LSTM Loss Over Epochs to show how the model improves during training.
CNN Accuracy Over Epochs to track performance across epochs.



In [ ]:
# ACD + ABSA Full Pipeline (SVM, LSTM, and CNN Models) - Optimized for Speed

from pyspark.sql.functions import col, array_contains
from pyspark.ml.feature import Tokenizer, HashingTF, IDF
from pyspark.ml.classification import LinearSVC
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense, Dropout, Conv1D, GlobalMaxPooling1D
import numpy as np
import tensorflow as tf
import matplotlib.pyplot as plt

# Step 1: Aspect Category Detection (ACD) with Clean Column Names
for aspect in all_categories:
    clean_aspect = aspect.replace('-', '_').replace('.', '_')
    df_with_categories = df_with_categories.withColumn(
        f"has_{clean_aspect}", array_contains(col("categories"), aspect).cast("integer")
    )

# Step 2: ABSA (SVM Classifier for each Aspect) with Caching for Speed
def train_absa_svm(aspect, df_with_categories):
    clean_aspect = aspect.replace('-', '_').replace('.', '_')
    aspect_df = df_with_categories.filter(col(f'has_{clean_aspect}') == 1).cache()

    tokenizer = Tokenizer(inputCol="text", outputCol="words")
    wordsData = tokenizer.transform(aspect_df)

    hashingTF = HashingTF(inputCol="words", outputCol="rawFeatures")
    featurizedData = hashingTF.transform(wordsData)

    idf = IDF(inputCol="rawFeatures", outputCol="features")
    tfidf_data = idf.fit(featurizedData).transform(featurizedData)

    svm = LinearSVC(featuresCol="features", labelCol="sentiment", maxIter=5)  # Reduced iterations for speed
    model = svm.fit(tfidf_data)

    aspect_df.unpersist()
    return model

# Step 3: Convert to Pandas for Deep Learning
train_pandas = train_data.limit(1000).toPandas()  # Limit data for speed
test_pandas = test_data.limit(200).toPandas()

# Convert sparse vector to numpy array
def sparse_to_array(v):
    return v.toArray() if hasattr(v, "toArray") else v

train_pandas['features_array'] = train_pandas['features'].apply(sparse_to_array)
test_pandas['features_array'] = test_pandas['features'].apply(sparse_to_array)

# Step 4: LSTM Model for ABSA with Reduced Complexity
X_train = np.stack(train_pandas['features_array'].values)
y_train = train_pandas[label_cols].values

X_test = np.stack(test_pandas['features_array'].values)
y_test = test_pandas[label_cols].values

lstm_model = Sequential([
    LSTM(64, input_shape=(X_train.shape[1], 1), return_sequences=False),  # Reduced LSTM units
    Dropout(0.2),
    Dense(len(label_cols), activation='sigmoid')
])

lstm_model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])
history_lstm = lstm_model.fit(X_train, y_train, epochs=5, batch_size=32, verbose=0)  # Fewer epochs

# Step 5: CNN Model for ABSA with Faster Conv1D
cnn_model = Sequential([
    Conv1D(32, 3, activation='relu', input_shape=(X_train.shape[1], 1)),  # Reduced filters
    GlobalMaxPooling1D(),
    Dense(len(label_cols), activation='sigmoid')
])

cnn_model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])
history_cnn = cnn_model.fit(X_train, y_train, epochs=5, batch_size=32, verbose=0)

# Step 6: Compare Models
svm_model = train_absa_svm("battery_life", df_with_categories)

print("SVM Model Trained")
print("LSTM Accuracy:", lstm_model.evaluate(X_test, y_test, verbose=0))
print("CNN Accuracy:", cnn_model.evaluate(X_test, y_test, verbose=0))

# Step 7: Visualizations
plt.figure(figsize=(10, 5))

# Plot LSTM Loss
plt.subplot(1, 2, 1)
plt.plot(history_lstm.history['loss'], label='LSTM Loss')
plt.title('LSTM Loss Over Epochs')
plt.xlabel('Epochs')
plt.ylabel('Loss')
plt.legend()

# Plot CNN Accuracy
plt.subplot(1, 2, 2)
plt.plot(history_cnn.history['accuracy'], label='CNN Accuracy')
plt.title('CNN Accuracy Over Epochs')
plt.xlabel('Epochs')
plt.ylabel('Accuracy')
plt.legend()

plt.tight_layout()
plt.show()


THE END

In [ ]:
# ACD + ABSA Full Pipeline (SVM, LSTM, and CNN Models) - Optimized for Speed

from pyspark.sql.functions import col, array_contains
from pyspark.ml.feature import Tokenizer, HashingTF, IDF
from pyspark.ml.classification import LinearSVC
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense, Dropout, Conv1D, GlobalMaxPooling1D
import numpy as np
import tensorflow as tf
import matplotlib.pyplot as plt

# Step 1: Aspect Category Detection (ACD)
for aspect in all_categories:
    df_with_categories = df_with_categories.withColumn(
        f"has_{aspect}", array_contains(col("categories"), aspect).cast("integer")
    )

# Step 2: ABSA (SVM Classifier for each Aspect) with Caching for Speed
def train_absa_svm(aspect, df_with_categories):
    aspect_df = df_with_categories.filter(col(f'has_{aspect}') == 1).cache()

    tokenizer = Tokenizer(inputCol="text", outputCol="words")
    wordsData = tokenizer.transform(aspect_df)

    hashingTF = HashingTF(inputCol="words", outputCol="rawFeatures")
    featurizedData = hashingTF.transform(wordsData)

    idf = IDF(inputCol="rawFeatures", outputCol="features")
    tfidf_data = idf.fit(featurizedData).transform(featurizedData)

    svm = LinearSVC(featuresCol="features", labelCol="sentiment", maxIter=5)  # Reduced iterations for speed
    model = svm.fit(tfidf_data)

    aspect_df.unpersist()
    return model

# Step 3: Convert to Pandas for Deep Learning
train_pandas = train_data.limit(1000).toPandas()  # Limit data for speed
test_pandas = test_data.limit(200).toPandas()

# Convert sparse vector to numpy array
def sparse_to_array(v):
    return v.toArray() if hasattr(v, "toArray") else v

train_pandas['features_array'] = train_pandas['features'].apply(sparse_to_array)
test_pandas['features_array'] = test_pandas['features'].apply(sparse_to_array)

# Step 4: LSTM Model for ABSA with Reduced Complexity
X_train = np.stack(train_pandas['features_array'].values)
y_train = train_pandas[label_cols].values

X_test = np.stack(test_pandas['features_array'].values)
y_test = test_pandas[label_cols].values

lstm_model = Sequential([
    LSTM(64, input_shape=(X_train.shape[1], 1), return_sequences=False),  # Reduced LSTM units
    Dropout(0.2),
    Dense(len(label_cols), activation='sigmoid')
])

lstm_model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])
history_lstm = lstm_model.fit(X_train, y_train, epochs=5, batch_size=32, verbose=0)  # Fewer epochs

# Step 5: CNN Model for ABSA with Faster Conv1D
cnn_model = Sequential([
    Conv1D(32, 3, activation='relu', input_shape=(X_train.shape[1], 1)),  # Reduced filters
    GlobalMaxPooling1D(),
    Dense(len(label_cols), activation='sigmoid')
])

cnn_model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])
history_cnn = cnn_model.fit(X_train, y_train, epochs=5, batch_size=32, verbose=0)

# Step 6: Compare Models
svm_model = train_absa_svm("battery-life", df_with_categories)

print("SVM Model Trained")
print("LSTM Accuracy:", lstm_model.evaluate(X_test, y_test, verbose=0))
print("CNN Accuracy:", cnn_model.evaluate(X_test, y_test, verbose=0))

# Step 7: Visualizations
plt.figure(figsize=(10, 5))

# Plot LSTM Loss
plt.subplot(1, 2, 1)
plt.plot(history_lstm.history['loss'], label='LSTM Loss')
plt.title('LSTM Loss Over Epochs')
plt.xlabel('Epochs')
plt.ylabel('Loss')
plt.legend()

# Plot CNN Accuracy
plt.subplot(1, 2, 2)
plt.plot(history_cnn.history['accuracy'], label='CNN Accuracy')
plt.title('CNN Accuracy Over Epochs')
plt.xlabel('Epochs')
plt.ylabel('Accuracy')
plt.legend()

plt.tight_layout()
plt.show()


Additional charts, such as confusion matrices or precision-recall curves.


Models were running too slow.
\Caching the DataFrame during SVM training to avoid redundant data processing.
Reducing the number of SVM iterations and limiting the dataset size for deep learning.
Simplifying the LSTM and CNN architectures by reducing the number of units and filters.
Lowering the number of epochs and silencing verbose outputs for faster training.

In [ ]:
# Convert Spark DataFrame to Pandas for deep learning
# Collect the data (be careful with large datasets)
train_pandas = train_data.toPandas()
test_pandas = test_data.toPandas()

# We need to convert the sparse vector features to numpy arrays
from pyspark.ml.linalg import SparseVector

def sparse_to_array(sparse_vector):
    return sparse_vector.toArray() if isinstance(sparse_vector, SparseVector) else sparse_vector

# Convert features to numpy arrays
train_pandas['features_array'] = train_pandas['features'].apply(sparse_to_array)
test_pandas['features_array'] = test_pandas['features'].apply(sparse_to_array)

# Prepare X (features) and y (labels) for training
X_train = np.stack(train_pandas['features_array'].values)
X_test = np.stack(test_pandas['features_array'].values)

# Get all label columns
label_cols = [col for col in train_pandas.columns if col.startswith('has_')]
y_train = train_pandas[label_cols].values
y_test = test_pandas[label_cols].values

# Now build a deep learning model
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout

# Define model architecture
model = Sequential([
    Dense(128, activation='relu', input_shape=(X_train.shape[1],)),
    Dropout(0.3),
    Dense(64, activation='relu'),
    Dropout(0.3),
    Dense(len(label_cols), activation='sigmoid')
])

# Compile model
model.compile(
    optimizer='adam',
    loss='binary_crossentropy',
    metrics=['accuracy', tf.keras.metrics.Precision(), tf.keras.metrics.Recall()]
)

# Train model
history = model.fit(
    X_train, y_train,
    validation_split=0.2,
    epochs=10,
    batch_size=32
)

# Evaluate on test set
results = model.evaluate(X_test, y_test)
print(f"Test Loss: {results[0]}")
print(f"Test Accuracy: {results[1]}")
print(f"Test Precision: {results[2]}")
print(f"Test Recall: {results[3]}")

# Make predictions
y_pred = model.predict(X_test)
y_pred_binary = (y_pred > 0.5).astype(int)

# Calculate F1 score manually
from sklearn.metrics import f1_score, precision_score, recall_score
precision = precision_score(y_test, y_pred_binary, average='micro')
recall = recall_score(y_test, y_pred_binary, average='micro')
f1 = f1_score(y_test, y_pred_binary, average='micro')

print(f"Precision: {precision}")
print(f"Recall: {recall}")
print(f"F1 Score: {f1}")

END WORKING *AREA*


In [ ]:
from pyspark.ml.classification import LinearSVC
from pyspark.ml.evaluation import MulticlassClassificationEvaluator
from pyspark.ml import Pipeline
from pyspark.ml.feature import VectorAssembler


# For simplicity, let's implement a binary classification model for each category
# In practice, you might want to use a multi-label approach

# Function to train and evaluate an SVM model for one category
def train_svm_for_category(category, train_data, test_data):
    # Prepare label column name
    label_col = f"has_{category.replace('-', '_').replace('.', '_')}"

    # Create SVM model
    svm = LinearSVC(featuresCol="features", labelCol=label_col, maxIter=10)

    # Train the model
    model = svm.fit(train_data)

    # Make predictions
    predictions = model.transform(test_data)

    # Evaluate model
   # evaluator = MulticlassClassificationEvaluator(
   #     labelCol=label_col, predictionCol="prediction", metricName="f1")
   # f1 = evaluator.evaluate(predictions)
    #
    evaluator = MulticlassClassificationEvaluator(labelCol=label_col,
                                              predictionCol="prediction",
                                              metricName="weightedPrecision")
precision = evaluator.evaluate(predictions)
print("Weighted Precision:", precision)
    evaluator.setMetricName("precision")
    precision = evaluator.evaluate(predictions)

    evaluator.setMetricName("recall")
    recall = evaluator.evaluate(predictions)

    return {"category": category, "precision": precision, "recall": recall, "f1": f1}

# Train models for all categories
svm_results = []
for category in all_categories:
    result = train_svm_for_category(category, train_data, test_data)
    svm_results.append(result)

# Print results
svm_results_df = spark.createDataFrame(svm_results)
svm_results_df.show()

In [ ]:
print("Training SVM models for all categories...")
svm_results = []
for category in all_categories:
    try:
        result = train_svm_for_category(category, train_data, test_data)
        svm_results.append(result)
        print(f"Completed category: {category}")
    except Exception as e:
        print(f"Error processing category {category}: {str(e)}")

# Create a dataframe with results
svm_results_df = spark.createDataFrame(svm_results)
print("SVM Results:")
svm_results_df.show()

# Calculate average metrics
avg_precision = np.mean([r["precision"] for r in svm_results])
avg_recall = np.mean([r["recall"] for r in svm_results])
avg_f1 = np.mean([r["f1"] for r in svm_results])

print(f"Average Precision: {avg_precision:.4f}")
print(f"Average Recall: {avg_recall:.4f}")
print(f"Average F1: {avg_f1:.4f}")

# Convert to pandas DataFrame for deep learning model
# We'll process a subset of records to handle memory constraints
# For a real model, you might want to use a data loader or mini-batches
print("Preparing data for deep learning model...")
train_pandas = train_data.limit(1000).toPandas()  # Limit to 1000 samples for demonstration
test_pandas = test_data.limit(200).toPandas()

# Function to convert sparse vector to array
def sparse_to_array(v):
    if hasattr(v, 'toArray'):
        return v.toArray()
    return v

# Apply conversion function
train_pandas['features_array'] = train_pandas['features'].apply(sparse_to_array)
test_pandas['features_array'] = test_pandas['features'].apply(sparse_to_array)

# Stack features into numpy arrays
X_train = np.stack(train_pandas['features_array'].values)
X_test = np.stack(test_pandas['features_array'].values)

# Extract labels
y_train = train_pandas[feature_cols].values
y_test = test_pandas[feature_cols].values

print("Data preparation for deep learning complete!")

In [ ]:
# Train models for all categories
svm_results = []
for category in all_categories:
   # result = train_svm_for_category(category, train_data, test_data)
    result = train_svm_for_category(category, train_data, test_data)
    svm_results.append(result)

# Print results
svm_results_df = spark.createDataFrame(svm_results)
svm_results_df.show()

Speeded up version

In [ ]:
# ACD + ABSA Full Pipeline (SVM, LSTM, and CNN Models) - Optimized for Speed

from pyspark.sql.functions import col, array_contains
from pyspark.ml.feature import Tokenizer, HashingTF, IDF
from pyspark.ml.classification import LinearSVC
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense, Dropout, Conv1D, GlobalMaxPooling1D
import numpy as np
import tensorflow as tf

# Step 1: Aspect Category Detection (ACD) - Clean column names
for aspect in all_categories:
    clean_aspect = aspect.replace('-', '_').replace('.', '_')
    df_with_categories = df_with_categories.withColumn(
        f"has_{clean_aspect}", array_contains(col("categories"), aspect).cast("integer")
    )

## Step 1: Aspect Category Detection (ACD)
#for aspect in all_categories:
#    df_with_categories = df_with_categories.withColumn(
#        f"has_{aspect}", array_contains(col("categories"), aspect).cast("integer")
#    )
#
#
## Step 2: ABSA (SVM Classifier for each Aspect) with Caching for Speed
#def train_absa_svm(aspect, df_with_categories):
#    aspect_df = df_with_categories.filter(col(f'has_{aspect}') == 1).cache()
#

# Step 2: Adjust the SVM Function to Use the Clean Column Name
def train_absa_svm(aspect, df_with_categories):
    clean_aspect = aspect.replace('-', '_').replace('.', '_')
    aspect_df = df_with_categories.filter(col(f'has_{clean_aspect}') == 1).cache()


    tokenizer = Tokenizer(inputCol="text", outputCol="words")
    wordsData = tokenizer.transform(aspect_df)

    hashingTF = HashingTF(inputCol="words", outputCol="rawFeatures")
    featurizedData = hashingTF.transform(wordsData)

    idf = IDF(inputCol="rawFeatures", outputCol="features")
    tfidf_data = idf.fit(featurizedData).transform(featurizedData)

    svm = LinearSVC(featuresCol="features", labelCol="sentiment", maxIter=5)  # Reduced iterations for speed
    model = svm.fit(tfidf_data)

    aspect_df.unpersist()
    return model

# Step 3: Convert to Pandas for Deep Learning
train_pandas = train_data.limit(1000).toPandas()  # Limit data for speed
test_pandas = test_data.limit(200).toPandas()

# Convert sparse vector to numpy array
def sparse_to_array(v):
    return v.toArray() if hasattr(v, "toArray") else v

train_pandas['features_array'] = train_pandas['features'].apply(sparse_to_array)
test_pandas['features_array'] = test_pandas['features'].apply(sparse_to_array)

# Step 4: LSTM Model for ABSA with Reduced Complexity
X_train = np.stack(train_pandas['features_array'].values)
y_train = train_pandas[label_cols].values

X_test = np.stack(test_pandas['features_array'].values)
y_test = test_pandas[label_cols].values

lstm_model = Sequential([
    LSTM(64, input_shape=(X_train.shape[1], 1), return_sequences=False),  # Reduced LSTM units
    Dropout(0.2),
    Dense(len(label_cols), activation='sigmoid')
])

lstm_model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])
lstm_model.fit(X_train, y_train, epochs=5, batch_size=32, verbose=0)  # Fewer epochs

# Step 5: CNN Model for ABSA with Faster Conv1D
cnn_model = Sequential([
    Conv1D(32, 3, activation='relu', input_shape=(X_train.shape[1], 1)),  # Reduced filters
    GlobalMaxPooling1D(),
    Dense(len(label_cols), activation='sigmoid')
])

cnn_model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])
cnn_model.fit(X_train, y_train, epochs=5, batch_size=32, verbose=0)

# Step 6: Compare Models
svm_model = train_absa_svm("battery-life", df_with_categories)

print("SVM Model Trained")
print("LSTM Accuracy:", lstm_model.evaluate(X_test, y_test, verbose=0))
print("CNN Accuracy:", cnn_model.evaluate(X_test, y_test, verbose=0))

Very slow version

In [ ]:
 #ACD + ABSA Full Pipeline (SVM, LSTM, and CNN Models)

from pyspark.sql.functions import col, array_contains
from pyspark.ml.feature import Tokenizer, HashingTF, IDF
from pyspark.ml.classification import LinearSVC
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense, Dropout, Conv1D, GlobalMaxPooling1D
import numpy as np
import tensorflow as tf

# Step 1: Aspect Category Detection (ACD)
# Binary indicator columns for each aspect
for aspect in all_categories:
    df_with_categories = df_with_categories.withColumn(
        f"has_{aspect}", array_contains(col("categories"), aspect).cast("integer")
    )

# Step 2: ABSA (SVM Classifier for each Aspect)
def train_absa_svm(aspect, df_with_categories):
    aspect_df = df_with_categories.filter(col(f'has_{aspect}') == 1)

    tokenizer = Tokenizer(inputCol="text", outputCol="words")
    wordsData = tokenizer.transform(aspect_df)

    hashingTF = HashingTF(inputCol="words", outputCol="rawFeatures")
    featurizedData = hashingTF.transform(wordsData)

    idf = IDF(inputCol="rawFeatures", outputCol="features")
    tfidf_data = idf.fit(featurizedData).transform(featurizedData)

    svm = LinearSVC(featuresCol="features", labelCol="sentiment", maxIter=10)
    model = svm.fit(tfidf_data)
    return model

# Step 3: Convert to Pandas for Deep Learning
train_pandas = train_data.toPandas()
test_pandas = test_data.toPandas()

# Convert sparse vector to numpy array
def sparse_to_array(v):
    return v.toArray() if hasattr(v, "toArray") else v

train_pandas['features_array'] = train_pandas['features'].apply(sparse_to_array)
test_pandas['features_array'] = test_pandas['features'].apply(sparse_to_array)

# Step 4: LSTM Model for ABSA
X_train = np.stack(train_pandas['features_array'].values)
y_train = train_pandas[label_cols].values

X_test = np.stack(test_pandas['features_array'].values)
y_test = test_pandas[label_cols].values

lstm_model = Sequential([
    LSTM(128, return_sequences=True, input_shape=(X_train.shape[1], 1)),
    Dropout(0.2),
    LSTM(64),
    Dense(len(label_cols), activation='sigmoid')
])

lstm_model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])
lstm_model.fit(X_train, y_train, epochs=10, batch_size=32)

# Step 5: CNN Model for ABSA
cnn_model = Sequential([
    Conv1D(64, 3, activation='relu', input_shape=(X_train.shape[1], 1)),
    GlobalMaxPooling1D(),
    Dense(64, activation='relu'),
    Dense(len(label_cols), activation='sigmoid')
])

cnn_model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])
cnn_model.fit(X_train, y_train, epochs=10, batch_size=32)

# Step 6: Compare SVM, LSTM, and CNN Models
svm_model = train_absa_svm("battery-life", df_with_categories)

print("SVM Model Trained")
print("LSTM Accuracy:", lstm_model.evaluate(X_test, y_test))
print("CNN Accuracy:", cnn_model.evaluate(X_test, y_test))


In [ ]:
#Old version of file loading into pandas


In [ ]:
import pyarrow.parquet as pq

# Try with pyarrow first to validate the file
try:
    dfp_train = pq.read_table(parquet_train_file)
    dfp_test = pq.read_table(parquet_train_file)
    dfp_validation = pq.read_table(parquet_validation_file)
    print("File can be read with pyarrow")
except Exception as e:
    print(f"Pyarrow error: {e}")